In [8]:
%load_ext autoreload
%autoreload 2

import metapack as mp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display 

from publicdata.census.files.metafiles import TableLookup
import shelve 

%matplotlib inline

mp.jupyter.init()


In [9]:
#pkg = mp.jupyter.open_package()
pkg = mp.jupyter.open_source_package()

# Setup importing form this package
pkg.set_sys_path()
import pylib # Import the code for the package
from pylib.census_path_processing import * 

pkg

# Metadata for the 2020 ACS
`civicknowledge.com-census_meta-2020e5-2.3.1` Last Update: 2023-03-28T19:46:01

_Deatilsed metadata for ACS releases._




 

## Contacts

* **Wrangler** [Eric Busboom](mailto:eric@civicknowledge.com), [Civic Knowledge](http://civicknowledge.com)

## Resources

* **[metadata](notebooks/build.ipynb#mdf)**. Combined columns and tables
* **[tables](notebooks/build.ipynb#tdf)**. 
* **[columns](notebooks/build.ipynb#cdf)**.

In [10]:
year = 2020
release = 5

tl = TableLookup(year, release)

In [15]:
tdf = tl.tables_df
tdf.head().T

,0,1,2,3,4
table_id,B01001,B01001A,B01001B,B01001C,B01001D
seq,[1],[1],[1],[1],[1]
start_pos,7,56,87,118,149
title,Sex By Age,Sex By Age (White Alone),Sex By Age (Black Or African American Alone),Sex By Age (American Indian And Alaska Native ...,Sex By Age (Asian Alone)
universe,Total population,People who are White alone,Black or African American alone,People who are American Indian and Alaska Nati...,People who are Asian alone
subject,Age-Sex,Age-Sex,Age-Sex,Age-Sex,Age-Sex
race,all,white,black,aian,asian
age_range,None,None,None,None,None
age,all,all,all,all,all
sex,all,all,all,all,all


In [18]:
cdf = tl.columns_df
cdf.head(6).iloc[:20].T

,0,1,2,3,4,5
uid,B01001_001,B01001_002,B01001_003,B01001_004,B01001_005,B01001_006
table_id,B01001,B01001,B01001,B01001,B01001,B01001
name,Total,Male,Under 5 years,5 to 9 years,10 to 14 years,15 to 17 years
sex,all,male,male,male,male,male
raceeth,all,all,all,all,all,all
age,all,all,000-005,005-009,010-014,015-017
age_range,None,None,"(0, 5)","(5, 9)","(10, 14)","(15, 17)"
min_age,0,0,0,5,10,15
max_age,120,120,5,9,14,17
poverty_status,all,all,all,all,all,all


In [24]:
cdf[cdf.filtered_pathname!=''].sample(10).T

,6818,3637,5371,15841,6324,8217,2482,1646,14533,19537
uid,B11003_005,B07404F_003,B08406_042,B24041_005,B09010_003,C15002B_009,B07001_029,B05008_039,B23001_126,B24123_435
table_id,B11003,B07404F,B08406,B24041,B09010,C15002B,B07001,B05008,B23001,B24123
name,Under 6 years and 6 to 17 years,Moved within same county,Public transportation (excluding taxicab),Construction,In family households,High school graduate (includes equivalency),60 to 64 years,Entered 2000 to 2009,In Armed Forces,"First-line supervisors of mechanics, installer..."
sex,all,all,female,all,all,female,all,female,female,all
raceeth,all,other,all,all,all,black,all,all,all,all
age,006-017,all,all,all,all,all,060-064,all,035-044,all
age_range,"(6, 17)",None,None,None,None,None,"(60, 64)",None,"(35, 44)",None
min_age,6,0,0,0,0,0,60,0,35,0
max_age,17,120,120,120,120,120,64,120,44,120
poverty_status,all,all,all,all,all,all,all,all,all,all


In [34]:
cdf[cdf.n_levels==0]

,uid,table_id,name,sex,raceeth,age,age_range,min_age,max_age,poverty_status,path,filtered_path,filtered_pathname,n_levels,level_1,level_2,level_3,level_4
0,B01001_001,B01001,Total,all,all,all,None,0,120,all,/total,None,,0,None,None,None,None
1,B01001_002,B01001,Male,male,all,all,None,0,120,all,/total/male,None,,0,None,None,None,None
2,B01001_003,B01001,Under 5 years,male,all,000-005,"(0, 5)",0,5,all,/total/male/under 5 years,None,,0,None,None,None,None
3,B01001_004,B01001,5 to 9 years,male,all,005-009,"(5, 9)",5,9,all,/total/male/5 to 9 years,None,,0,None,None,None,None
4,B01001_005,B01001,10 to 14 years,male,all,010-014,"(10, 14)",10,14,all,/total/male/10 to 14 years,None,,0,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27834,B99281_002,B99281,Allocated,all,all,all,None,0,120,all,/total/allocated,None,/allocated,0,None,None,None,None
27835,B99281_003,B99281,Not allocated,all,all,all,None,0,120,all,/total/not allocated,None,/not allocated,0,None,None,None,None
27836,B99282_001,B99282,Total,all,all,all,None,0,120,all,/total,None,,0,None,None,None,None
27845,B99283_001,B99283,Total,all,all,all,None,0,120,all,/total,None,,0,None,None,None,None


In [42]:
t = cdf[cdf.n_levels==1]
for gn, g in t.groupby('table_id'):
    if g.level_1.nunique() > 1:
        print(gn, g.level_1.unique())


B02014 ['american indian tribes, specified' 'alaska native tribes, specified']
B02016 ['polynesian' 'micronesian' 'melanesian']
B03002 ['not hispanic or latino' 'hispanic or latino']
B04004 ['arab' 'subsaharan african' 'west indian (except hispanic groups)']
B04005 ['arab' 'subsaharan african' 'west indian (except hispanic groups)']
B04006 ['arab' 'subsaharan african' 'west indian (except hispanic groups)']
B04007 ['ancestry specified' 'ancestry not specified']
B05002 ['native' 'foreign born']
B05005 ['entered 2010 or later' 'entered 2000 to 2009' 'entered 1990 to 1999'
 'entered before 1990']
B05005PR ['entered 2010 or later' 'entered 2000 to 2009' 'entered 1990 to 1999'
 'entered before 1990']
B05006 ['europe' 'asia' 'africa' 'oceania' 'americas']
B05006PR ['europe' 'asia' 'americas']
B05007 ['entered 2010 or later' 'entered 2000 to 2009' 'entered 1990 to 1999'
 'entered before 1990' 'europe' 'asia' 'latin america' 'other areas']
B05008 ['europe' 'asia' 'latin america' 'other areas']

In [46]:
cdf[~cdf.filtered_path.isnull()].filtered_path

367                      /two or more races
368                      /two or more races
377      /american indian tribes, specified
378      /american indian tribes, specified
379      /american indian tribes, specified
                        ...                
27843                       /other computer
27844                       /other computer
27846                            /allocated
27847                            /allocated
27848                            /allocated
Name: filtered_path, Length: 11261, dtype: object

In [11]:
tdf = tl.tables_df
cdf = tl.columns_df

cdf = cdf[['uid', 'table_id', 'name', 'sex', 'raceeth', 'age', 'poverty_status', 'filtered_pathname']]\
    .rename(columns={'filtered_pathname':'path', 'uid':'column_id'})

tdf = tdf[['table_id', 'title', 'bare_title', 'stripped_title','universe', 'subject', 'race',
       'age', 'sex']].copy().rename(columns={'race':'raceeth'})

tdf.head()

/Users/eric/proj/metatab-dev/metapack-dev/publicdata_census/src/publicdata/census/files/metafiles.py:567: FutureWarning: The default value of regex will change from True to False in a future version.
  df['stripped_title'] = df.stripped_title.str.replace(s, '')


,table_id,title,bare_title,stripped_title,universe,subject,raceeth,age,sex
0,B01001,Sex By Age,Sex By Age,Population,Total population,Age-Sex,all,all,all
1,B01001A,Sex By Age (White Alone),Sex By Age,Population,People who are White alone,Age-Sex,white,all,all
2,B01001B,Sex By Age (Black Or African American Alone),Sex By Age,Population,Black or African American alone,Age-Sex,black,all,all
3,B01001C,Sex By Age (American Indian And Alaska Native ...,Sex By Age,Population,People who are American Indian and Alaska Nati...,Age-Sex,aian,all,all
4,B01001D,Sex By Age (Asian Alone),Sex By Age,Population,People who are Asian alone,Age-Sex,asian,all,all


In [12]:
cdf.head()

,column_id,table_id,name,sex,raceeth,age,poverty_status,path
0,B01001_001,B01001,Total,all,all,all,all,
1,B01001_002,B01001,Male,male,all,all,all,
2,B01001_003,B01001,Under 5 years,male,all,000-005,all,
3,B01001_004,B01001,5 to 9 years,male,all,005-009,all,
4,B01001_005,B01001,10 to 14 years,male,all,010-014,all,


In [13]:

trename_map = { k:'table_'+k for k in 'raceeth age sex'.split()}
#crename_map = { k:'col_'+k for k in 'raceeth age sex'.split()}
mdf=tdf.rename(columns=trename_map).merge(cdf, on='table_id').copy()

def mk_update_f(table_col_name, col_name):

    def _update(r):
        """ If the column isn't restricted, but the table is, then the column should be marked as restricted."""
        if r[table_col_name] != 'all' and r[col_name] == 'all':
            r[col_name] = r[table_col_name]

        return r
    
    return _update

_sex = mk_update_f('table_sex','sex')
_age = mk_update_f('table_age','age')
_raceeth = mk_update_f('table_raceeth','raceeth')

mdf = pd.DataFrame([_raceeth(_age(_sex(r)))  for idx, r in mdf.iterrows()])


mdf[  (mdf.table_sex!='all') ].head().T

,14635,14636,14637,14638,14639
table_id,B23003,B23003,B23003,B23003,B23003
title,Presence Of Own Children Under 18 Years By Age...,Presence Of Own Children Under 18 Years By Age...,Presence Of Own Children Under 18 Years By Age...,Presence Of Own Children Under 18 Years By Age...,Presence Of Own Children Under 18 Years By Age...
bare_title,Presence Of Own Children Under 18 Years By Age...,Presence Of Own Children Under 18 Years By Age...,Presence Of Own Children Under 18 Years By Age...,Presence Of Own Children Under 18 Years By Age...,Presence Of Own Children Under 18 Years By Age...
stripped_title,Presence Of Own Children Under 18 Years By Age...,Presence Of Own Children Under 18 Years By Age...,Presence Of Own Children Under 18 Years By Age...,Presence Of Own Children Under 18 Years By Age...,Presence Of Own Children Under 18 Years By Age...
universe,Females 20 to 64 years in households,Females 20 to 64 years in households,Females 20 to 64 years in households,Females 20 to 64 years in households,Females 20 to 64 years in households
subject,Employment Status,Employment Status,Employment Status,Employment Status,Employment Status
table_raceeth,all,all,all,all,all
table_age,020-064,020-064,020-064,020-064,020-064
table_sex,female,female,female,female,female
column_id,B23003_001,B23003_002,B23003_003,B23003_004,B23003_005


In [14]:

from pylib.descriptions import description
mdf['description'] = mdf.apply(description, axis=1)

